In [1]:
%pip install langdetect nltk sentence_transformers

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 615.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 1.7 MB/s eta 0:00:0000:010:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.26.2 which is incompatible.


# Движок полнотекстового поиска

In [2]:
%pip install natasha

Defaulting to user installation because normal site-packages is not writeable


In [25]:
from langdetect import detect
from concurrent.futures import ThreadPoolExecutor
from nltk import SnowballStemmer
from tqdm import tqdm
from gensim.models import Word2Vec


class FullText:
    def __init__(self, data):
        self.idx = self.index_data(data)
        

    def get_stem(self, sentence: str) -> list[str]:
        language = 'russian'
        try:
            language = 'english' if detect(sentence) != 'ru' else 'russian'
        except:
            pass

        stemmer = SnowballStemmer(language)
        my_words = sentence.split()
        return [stemmer.stem(word) for word in my_words]

    def index_data(self, data):
        data_idx = {}
        
        for i in tqdm(range(0, 100000)):
            try:
                title = str(data.loc[i, 'video_title'])
                video_id = data.loc[i, 'video_id']

                stems = self.get_stem(title)
                for stem in stems:
                    if stem in data_idx:
                        data_idx[stem].append(video_id)
                    else:
                        data_idx[stem] = [video_id]
                            
            except TypeError:
                print("Huy")
                    
        return data_idx

    @staticmethod
    def transliterate(name):
        # Словарь с заменами
        slovar_1 = {'ай': 'i', 'а': 'a', 'б': 'b', 'в': 'v', 'г': 'g', 'дж': "j", 'д': 'd', 'ей': 'ay', 'е': 'e',
                    'ё': 'yo',
                    'ж': 'j', 'з': 'z', 'и': 'i', 'й': 'i', 'к': 'k', 'л': 'l', 'м': 'm', 'н': 'n',
                    'о': 'o', 'п': 'p', 'р': 'r', 'с': 's', 'т': 't', 'у': 'u', 'ф': 'f', 'х': 'h',
                    'ц': 'c', 'ч': 'ch', 'ш': 'sh', 'щ': 'sch', 'ъ': '', 'ы': 'y', 'ь': '', 'э': 'e',
                    'ю': 'u', 'я': 'ya'}

        slovar_2 = {'ай': 'i', 'а': 'a', 'б': 'b', 'в': 'v', 'г': 'g', 'дж': "j", 'д': 'd', 'ей': 'ay', 'е': 'e',
                    'ё': 'yo',
                    'ж': 'j', 'з': 'z', 'и': 'i', 'й': 'i', 'к': 'c', 'л': 'l', 'м': 'm', 'н': 'n',
                    'о': 'o', 'п': 'p', 'р': 'r', 'с': 's', 'т': 't', 'у': 'u', 'ф': 'f', 'х': 'h',
                    'ц': 'c', 'ч': 'ch', 'ш': 'sh', 'щ': 'sch', 'ъ': '', 'ы': 'y', 'ь': '', 'э': 'e',
                    'ю': 'u', 'я': 'ya'}

        name = name.lower()

        translit_1 = name
        for key in slovar_1:
            translit_1 = translit_1.replace(key, slovar_1[key])

        translit_2 = name
        for key in slovar_2:
            translit_2 = translit_2.replace(key, slovar_2[key])

        return [translit_1, translit_2]

    def search(self, query):
        stems = self.get_stem(query)
        result = []
        for stem in stems:
            if stem in self.idx:
                result += self.idx[stem]

        for translit in self.transliterate(query) :
            stems_t = self.get_stem(translit)
            for stem in stems_t:
                if stem in self.idx:
                    result += self.idx[stem]

        return set(result)


# Движок семантического поиска

In [4]:
import sentence_transformers
from sentence_transformers import SentenceTransformer


class Semantic:
    def __init__(self, sentence_model: SentenceTransformer, threshold=0.5):
        self.model = sentence_model
        self.threshold = threshold

    def get_semantic_list(self, query: str, data: list[str]):
        semantic_result = []

        query_sentence = self.model.encode(query)
        embeddings = self.model.encode(data)

        i = 0

        for sentence in embeddings:
            s = sentence_transformers.util.cos_sim(a=sentence, b=query_sentence)
            if s > self.threshold:
                semantic_result.append(data[i])

        return semantic_result


# Парсинг данных

In [32]:
import pandas as pd

data = pd.read_parquet(
    "videos.parquet",
    engine="fastparquet",
    columns=[
        "video_id",
        "video_title",
    ],
)

In [6]:
len(data.axes[0])

34404561

In [8]:
# Пример использования
import sentence_transformers

def find_by_key(iterable, key, value):
    for index, dict_ in enumerate(iterable):
        if key in dict_ and dict_[key] == value:
            return dict_


fulltext = FullText(data)

model = sentence_transformers.SentenceTransformer('inkoziev/sbert_pq' )
semantic = Semantic(model, 0.2)

100%|██████████| 100000/100000 [08:55<00:00, 186.86it/s]


.gitattributes:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.84k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/410 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [12]:
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Загрузка данных
df = pd.read_parquet(
    "features.parquet",
    engine="fastparquet",
    columns=[
        "video_id",
        "v_category",
        "v_pub_datetime",
        "v_likes",
        "v_dislikes",
        "total_comments",
        "v_cr_click_long_view_7_days"
    ],
)

In [ ]:
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Загрузка данных
df = pd.read_parquet(
    "features.parquet",
    engine="fastparquet",
    columns=[
        "video_id",
        "v_category",
        "v_pub_datetime",
        "v_likes",
        "v_dislikes",
        "total_comments",
        "v_cr_click_long_view_7_days"
    ],
)

# Функция для определения весов в зависимости от категории видео
def get_weights(category):
    if category in ["Телепередачи", "Спорт", "Спорт/Игры"]:
        return {"v_pub_datetime": 0.4, "v_dislikes": 0.25, "v_likes": 0.2,  "v_cr_click_long_view_7_days": 0.1, "total_comments": 0.1}
    else:
        return {"v_dislikes": 0.25, "v_likes": 0.3, "v_cr_click_long_view_7_days": 0.2, "total_comments": 0.1, "v_pub_datetime": 0.1}

# Определение весов для каждого фактора (важности)
df["weights"] = df["v_category"].apply(get_weights)

# Convert "v_pub_datetime" to numeric format
label_encoder = LabelEncoder()
df["v_pub_datetime"] = label_encoder.fit_transform(df["v_pub_datetime"])

# Нормализация данных
scaler = StandardScaler()
numeric_columns = ["v_pub_datetime", "v_likes", "v_dislikes", "v_cr_click_long_view_7_days", "total_comments", "v_category"]
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# Разделение данных на обучающую и тестовую выборки
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Создание Pool для обучения
train_pool = Pool(
    train_data[numeric_columns],
    label=train_data["video_id"],
    cat_features=["v_category"]
)

# Инициализация модели CatBoost
model = CatBoostRegressor(
    iterations=1000, depth=6, learning_rate=0.1, loss_function="RMSE"
)

# Обучение модели
model.fit(train_pool, verbose=100)

# Использование модели для ранжирования тестовых данных
test_pool = Pool(
    test_data[numeric_columns],
    cat_features=["v_category"]
)
test_data["predicted_rank"] = model.predict(test_pool)

# Сортировка тестовых данных по предсказанному рангу
test_data = test_data.sort_values(by="predicted_rank", ascending=False)
print(test_data[["video_id", "predicted_rank"]])


In [19]:
# Получаем метрику
queries = pd.read_csv("./test_dataset_submission_queries.csv")
queries

,query
0,Битва сильнейших экстрасенсов 2023 смотреть | ...
1,битва сильнейших экстрасенсов 2023\nбитва силь...
2,"Экстрасенсы. Битва сильнейших, 4 выпуск"
3,супер стар 4 сезон
4,пять ночей с Фредди
...,...
1995,Шоу аватар 2023
1996,Макс
1997,смотреть мужское женское 2019
1998,шоу вована и лексуса


In [ ]:
from tqdm import tqdm

resp_data = {
    'query': [],
    'video_id': [],
}


for index, row in tqdm(queries.iterrows()):
    query = row["query"]
    results = fulltext.search(query)
    
    response = {
        "video_id": [],
        "video_title": []
    }
    
    for res in results:
        row = data[data['video_id'] == res]['video_title'].to_string(index=False)
        response['video_id'].append(res)
        response['video_title'].append(row)
        
    results = semantic.get_semantic_list(query, response['title'])
    
    data = pd.DataFrame(response)

    for result in results:
        video_id = data[data['video_title'] == result]['video_id'].to_string(index=False)
        resp_data['query'].append(query)
        resp_data['video_id'].append(video_id)

0it [00:00, ?it/s]